In [ ]:
from tqdm import tqdm

In [ ]:
import utils as u

In [ ]:
def get_file_infos(filename,dl_path):
    f_s=dl_path.split('/')
    suborigin_=f_s[6]
    origin_=f_s[2]
    nature_=f_s[5]
    year_=f_s[3]
    
    import datetime
    from datetime import datetime

    f_s = filename.split('_')
    date_=f_s[2]
    year_=date_[0:4]
    month_=date_[4:6]
    day_=date_[6:9]
    time_= f_s[3]

    datetime_str=date_ + '_' + time_
    timestamp_=datetime.strptime(datetime_str,"%Y%m%d_%H%M%S")
    
    return (nature_,origin_,suborigin_,year_,timestamp_)

In [ ]:
    # Import S3 connectors librairies
    #from awsPy.aws_authorization import aws_connector
    import importlib.util
    spec = importlib.util.spec_from_file_location("aws_connector.py", "../../../../aws-python/awsPy/aws_authorization/aws_connector.py")
    aws_connector = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(aws_connector)

    #from awsPy.aws_s3 import service_s3
    import importlib.util
    spec = importlib.util.spec_from_file_location("service_s3.py", "../../../../aws-python/awsPy/aws_s3/service_s3.py")
    service_s3 = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(service_s3)

    # Connect to S3
    import os
    from pathlib import Path

    bucket = 'calfdata'
    path = os.getcwd()
    parent_path = str(Path(path).parent)
    path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

    con = aws_connector.aws_instantiate(credential = path_cred,
                                            region = 'eu-west-3')
    client= con.client_boto()
    s3 = service_s3.connect_S3(client = client,
                          bucket = 'calfdata')

    bucket_name='calfdata'
    bucket = client['resource'].Bucket(bucket_name)    

# From INPI/Flux folder

In [ ]:
#Dossier source à parcourir
prefix_root='INPI/TC_1/Flux'# Attention pas plus loin dans l'arborescence que 'INPI/TC_1/Flux/'
# Calcul de la liste des dossiers qui devront être traités

list_folders=[]
for y in range(2017, 2018):
    for m in range(1, 13):
        m_ = '0'+ str(m) if m<10 else str(m)
        for n in ['ETS']:#['ACTES','ETS','OBS','PM','PP','REP','COMPTES_ANNUELS']
            for s in ['EVT','NEW']:
                prefix_="{}/{}/{}/{}/{}".format(prefix_root,y,m_,n,s)
                list_folders.append(prefix_)
log = open('data/log-flux.txt', 'w+')

In [ ]:
# Recréation pour correction du Flux/2018/Actes/New
'''list_folders=['INPI/TC_1/Flux/2018/01/OBS/EVT',
             'INPI/TC_1/Flux/2018/02/OBS/EVT',
             'INPI/TC_1/Flux/2018/03/OBS/EVT',
              'INPI/TC_1/Flux/2018/04/OBS/EVT',
              'INPI/TC_1/Flux/2018/05/OBS/EVT',
              'INPI/TC_1/Flux/2018/06/OBS/EVT',
              'INPI/TC_1/Flux/2018/07/OBS/EVT',
              'INPI/TC_1/Flux/2018/08/OBS/EVT',
              'INPI/TC_1/Flux/2018/09/OBS/EVT',
              'INPI/TC_1/Flux/2018/10/OBS/EVT',
              'INPI/TC_1/Flux/2018/11/OBS/EVT',
              'INPI/TC_1/Flux/2018/12/OBS/EVT'
             ]
             
log = open('data/log-flux-2018-obsevt.txt', 'w+')             
'''

In [ ]:
import pandas as pd
import io
from io import StringIO

for prefix_ in list_folders:
    print('treat ' + prefix_)
    list_bucket = list(bucket.objects.filter(Prefix=prefix_))
    
    for obj in tqdm(list_bucket):
        
        try:
            key = obj.key
            body = obj.get()['Body'].read()
            print(key, file=log)

            s=key.split('/')
            filename=s[len(s)-1]

            (nature_,origin_,suborigin_,year_,timestamp_) = u.get_file_infos(filename)
            dest_path="{}/{}/{}/{}/{}".format('INPI/TC_1/01_donnee_source',origin_,year_,nature_,suborigin_)
            dest_full_path="{}/{}".format(dest_path,filename)
            
            
            print(filename)
            df = pd.read_csv(io.BytesIO(body), header=0, dtype=str, sep = ';',error_bad_lines=False)
            if nature_ == 'REP':
                # Rename col 'date_greffe' to 'Date_Greffe'
                df=df.rename(columns={"date_greffe": "Date_Greffe"})

            df['csv_source']=filename
            df['nature']=nature_
            df['type']=origin_
            df['origin']=suborigin_

            if origin_=='Stock' and year_ == '2017' and nature_ == 'ACTES': 
                # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
                df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
            elif origin_=='Stock' and year_ == '2017' and nature_ == 'COMPTES_ANNUELS': 
                # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
                df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
            elif origin_=='Stock' and year_ == '2017': 
                # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Greffe
                df['file_timestamp']= df['Date_Greffe'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
            else:
                df['file_timestamp']=timestamp_    

            # Save file to destination in S3
            # Create buffer
            csv_buffer = StringIO()
            # Write dataframe to buffer
            df.to_csv(csv_buffer, sep=";", index=False)
            # Create S3 object
            s3_resource = client['resource']
            # Write buffer to S3 object
            s3_resource.Object(bucket_name, dest_full_path).put(Body=csv_buffer.getvalue())
        except Exception as e:
            print(e)
log.close()

# From Raw Folder

In [ ]:
import pandas as pd
import io
from io import StringIO
import time

In [ ]:
# Initiate log files
def log_init():

    logv_name="{}/{}_{}-{}".format('data',timestr,folder_str,'viewed.csv')
    logv = open(logv_name, 'w')
    print('key', file=logv)
    logt_name="{}/{}_{}-{}".format('data',timestr,folder_str,'treated.csv')
    logt = open(logt_name, 'w')
    print('key', file=logt)
    loge_name="{}/{}_{}-{}".format('data',timestr,folder_str,'errors.csv')
    loge = open(loge_name, 'w')
    print('key;error', file=loge)    
    
    return (logtd,logv,logt,loge)

In [ ]:
def prepare_object(obj):
    key = obj.key
    body = obj.get()['Body'].read()
    s=key.split('/')
    filename=s[len(s)-1]

    (nature_,origin_,suborigin_,year_,timestamp_) = u.get_file_infos(filename)
    dest_path="{}/{}/{}/{}/{}".format('INPI/TC_1/01_donnee_source',origin_,year_,nature_,suborigin_)
    dest_full_path="{}/{}".format(dest_path,filename)
            
    df = pd.read_csv(io.BytesIO(body), header=0, dtype=str, sep = ';',error_bad_lines=False)
    if nature_ == 'REP':
        # Rename col 'date_greffe' to 'Date_Greffe'
        df=df.rename(columns={"date_greffe": "Date_Greffe"})

    df['csv_source']=filename
    df['nature']=nature_
    df['type']=origin_
    df['origin']=suborigin_

    if origin_=='Stock' and year_ == '2017' and nature_ == 'ACTES': 
        # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
        df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
    elif origin_=='Stock' and year_ == '2017' and nature_ == 'COMPTES_ANNUELS': 
        # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
        df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
    elif origin_=='Stock' and year_ == '2017': 
        # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Greffe
        df['file_timestamp']= df['Date_Greffe'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
    else:
        df['file_timestamp']=timestamp_    

    # Save file to destination in S3
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    df.to_csv(csv_buffer, sep=";", index=False)
    # Create S3 object
    s3_resource = client['resource']
    # Write buffer to S3 object
    s3_resource.Object(bucket_name, dest_full_path).put(Body=csv_buffer.getvalue())

    
    return dest_full_path

# Prepare all items in a folder

In [ ]:
# Select folder
folder='INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2018/01/01'
folder_str='rawFlux20180101'

In [ ]:
list_bucket = bucket.objects.filter(Prefix=folder)

In [ ]:
# Optional filter on certain filenames

filter_ = (
#'1_PM.csv',
#'2_PM_EVT.csv',
#'3_PP.csv',
#'4_PP_EVT.csv',
#'5_rep.csv',
#'6_rep_nouveau_modifie_EVT.csv',
#'7_rep_partant_EVT.csv',
#'8_ets.csv',
#'9_ets_nouveau_modifie_EVT.csv',
'10_ets_supprime_EVT.csv',
#'11_obs.csv',
'12_actes.csv',
'13_comptes_annuels.csv'
    )

list_bucket_filter=[]
for obj in tqdm(list_bucket):
    if obj.key.endswith(filter_):
        list_bucket_filter.append(obj)
        
list_bucket=list_bucket_filter if filter_ else list_bucket_filter

In [ ]:
# Save todo list to a file
import csv
timestr = time.strftime("%Y%m%d-%H%M%S")
logtd_name="{}/{}_{}-{}".format('data',timestr,folder_str,'todo.csv')
logtd = open(logtd_name, 'w')
with logtd as myfile:
    wr = csv.writer(myfile,delimiter=';',quoting=csv.QUOTE_ALL)
    wr.writerow(['key'])
    for obj in tqdm(list_bucket):
        wr.writerow([obj.key])
logtd.close()

In [ ]:
(logtd,logv,logt,loge)=log_init()

In [ ]:
for obj in tqdm(list_bucket):

    try:
        print(obj.key, file=logv)
        prepare_object(obj)
        print(obj.key, file=logt)
        
    except Exception as e:
        print((obj.key,e), file=loge)

logv.close()
logt.close()
loge.close()

# Stats and error recovery

In [ ]:
# Stats 
dftd=pd.read_csv(logtd.name,header=0, sep=';')# Todo
td=len(dftd)
dft=pd.read_csv(logt.name,header=0, sep=';')
t=len(dft)
dfe=pd.read_csv(loge.name,header=0, sep=';',usecols = ['key'])
e=len(dfe)
# Find untreated objects
dfnt=pd.DataFrame()
dfnt=dftd.merge(dft,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
dfnt=dfnt.drop(columns=['_merge'])
nt=len(dfnt)
# Add errors
dfnt=dfnt.append(dfe,sort=True)
nt=len(dfnt)



# Print stats
print('Folder :' + folder)
if filter:
    print("Filter : %s" % (filter_,))
print('To do : ' + str(td))
print('Treated : ' + str(t))
print('Errors : ' + str(e))
print('Retake : ' + str(nt))

# Prepare from a file list

In [ ]:
# Save errors to a new todo
list_bucket_keys = dfnt['key'].values.tolist()

In [ ]:
# Or Create the new todo from a list of Missing files
folder_str='prepFluxMissing2018'
df=pd.read_csv('data/count/missingPrepFlux2018.csv',header=None, sep=';')# Todo
df.columns = ['key']
list_bucket_keys = df['key'].values.tolist()

In [ ]:
# Save list to a new todo
import csv
timestr = time.strftime("%Y%m%d-%H%M%S")
logtd_name="{}/{}_{}-{}".format('data',timestr,folder_str,'todo.csv')
logtd = open(logtd_name, 'w')
with logtd as myfile:
    wr = csv.writer(myfile,delimiter=';',quoting=csv.QUOTE_ALL)
    wr.writerow(['key'])
    for key in tqdm(list_bucket_keys):
        wr.writerow([key])
logtd.close()

In [ ]:
(logtd,logv,logt,loge)=log_init()

In [ ]:
# Restart on errors
for n,key in tqdm(enumerate(list_bucket_keys)):
    try:
        print(key, file=logv)
        
        # get object in S3 by its key
        obj=client['resource'].Object(bucket_name=bucket_name, key=key)
        f=prepare_object(obj)
        print(key, file=logt)
        
    except Exception as e:
        print((key,e), file=loge)

logv.close()
logt.close()
loge.close()

In [ ]:
# 2017 : 31 520 fichiers ~ 1h - 4 erreurs à traiter 20200412-225237_prepFluxMissing2017-errors.csv

In [ ]:
# 2018 : 137 752 fichiers ~ 4h40?